<a href="https://colab.research.google.com/github/orutkina/-./blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas seaborn scikit-learn

# Домашнее задание про практике 2

In [ ]:
# Импорт всех необходимых библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Установка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

Часть 1: Загрузка и предварительный анализ данных

In [ ]:
# Загрузка данных
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')

# Первичный осмотр данных
print("Размер датасета:", df.shape)
print("\nПервые 5 строк:")
print(df.head())
print("\nИнформация о данных:")
print(df.info())
print("\nСтатистическое описание:")
print(df.describe())
print("\nПроверка на пропуски:")
print(df.isnull().sum())

Часть 2: Предобработка данных

In [ ]:
# Копируем датасет для работы
df_clean = df.copy()

# 1. Проверяем и обрабатываем пропуски
missing_values = df_clean.isnull().sum()
print("Пропущенные значения:\n", missing_values[missing_values > 0])

# В этом датасете обычно нет пропусков, но если бы были:
# Для числовых признаков - заполняем медианой
# Для категориальных - заполняем модой
# В данном случае просто пропускаем этот шаг

# 2. Преобразуем типы данных
# Определяем категориальные признаки
categorical_cols = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']
for col in categorical_cols:
    df_clean[col] = df_clean[col].astype('category')

# 3. Выбираем целевую переменную
# Я выберу 'Sleep Duration' (продолжительность сна) как целевую переменную
# Это непрерывная числовая переменная, подходящая для регрессии
y = df_clean['Sleep Duration']
X = df_clean.drop('Sleep Duration', axis=1)

print(f"\nЦелевая переменная: Sleep Duration")
print(f"Размер X: {X.shape}, Размер y: {y.shape}")

Часть 3: Разведочный анализ данных (EDA)

In [ ]:
# Создаем фигуру для визуализации
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Разведочный анализ данных (EDA)', fontsize=16, fontweight='bold')

# 1. Распределение целевой переменной
axes[0, 0].hist(y, bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Sleep Duration (часы)')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].set_title('Распределение продолжительности сна')
axes[0, 0].axvline(y.mean(), color='red', linestyle='--', label=f'Среднее: {y.mean():.2f}')
axes[0, 0].axvline(y.median(), color='green', linestyle='--', label=f'Медиана: {y.median():.2f}')
axes[0, 0].legend()

# 2. Взаимосвязь с возрастом
axes[0, 1].scatter(df_clean['Age'], y, alpha=0.6)
axes[0, 1].set_xlabel('Возраст')
axes[0, 1].set_ylabel('Продолжительность сна')
axes[0, 1].set_title('Сон vs Возраст')
z = np.polyfit(df_clean['Age'], y, 1)
p = np.poly1d(z)
axes[0, 1].plot(df_clean['Age'], p(df_clean['Age']), "r--", alpha=0.8)

# 3. Взаимосвязь с физической активностью
axes[0, 2].scatter(df_clean['Physical Activity Level'], y, alpha=0.6)
axes[0, 2].set_xlabel('Уровень физической активности')
axes[0, 2].set_ylabel('Продолжительность сна')
axes[0, 2].set_title('Сон vs Физическая активность')
z = np.polyfit(df_clean['Physical Activity Level'], y, 1)
p = np.poly1d(z)
axes[0, 2].plot(df_clean['Physical Activity Level'], p(df_clean['Physical Activity Level']), "r--", alpha=0.8)

# 4. Взаимосвязь с уровнем стресса
axes[1, 0].scatter(df_clean['Stress Level'], y, alpha=0.6)
axes[1, 0].set_xlabel('Уровень стресса')
axes[1, 0].set_ylabel('Продолжительность сна')
axes[1, 0].set_title('Сон vs Уровень стресса')
z = np.polyfit(df_clean['Stress Level'], y, 1)
p = np.poly1d(z)
axes[1, 0].plot(df_clean['Stress Level'], p(df_clean['Stress Level']), "r--", alpha=0.8)

# 5. Сон по гендеру
gender_means = df_clean.groupby('Gender')['Sleep Duration'].mean()
axes[1, 1].bar(gender_means.index, gender_means.values)
axes[1, 1].set_xlabel('Пол')
axes[1, 1].set_ylabel('Средняя продолжительность сна')
axes[1, 1].set_title('Сон по гендеру')
for i, v in enumerate(gender_means.values):
    axes[1, 1].text(i, v + 0.05, f'{v:.2f}', ha='center')

# 6. Корреляционная матрица числовых признаков
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
corr_matrix = df_clean[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            ax=axes[1, 2], cbar_kws={'label': 'Коэффициент корреляции'})
axes[1, 2].set_title('Корреляционная матрица')

plt.tight_layout()
plt.show()

# Анализ категориальных признаков
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 1. Сон по категориям BMI
bmi_means = df_clean.groupby('BMI Category')['Sleep Duration'].mean().sort_values()
axes[0].barh(bmi_means.index, bmi_means.values)
axes[0].set_xlabel('Средняя продолжительность сна')
axes[0].set_title('Сон по категориям BMI')

# 2. Сон по профессиям (топ-10)
occupation_means = df_clean.groupby('Occupation')['Sleep Duration'].mean().sort_values(ascending=False)[:10]
axes[1].barh(occupation_means.index, occupation_means.values)
axes[1].set_xlabel('Средняя продолжительность сна')
axes[1].set_title('Топ-10 профессий по продолжительности сна')

plt.tight_layout()
plt.show()

Часть 4: Подготовка признаков для моделирования

In [ ]:
# Определяем категориальные и числовые признаки
categorical_features = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']
numeric_features = ['Age', 'Physical Activity Level', 'Stress Level',
                    'Heart Rate', 'Daily Steps', 'Systolic BP', 'Diastolic BP']

# Отбираем лучшие признаки на основе EDA
# Выбираем признаки с наибольшим влиянием на целевую переменную
best_features = ['Age', 'Physical Activity Level', 'Stress Level',
                 'Heart Rate', 'Daily Steps', 'BMI Category', 'Occupation']

print(f"Категориальные признаки: {categorical_features}")
print(f"Числовые признаки: {numeric_features}")
print(f"Лучшие признаки: {best_features}")

# Функция для подготовки данных
def prepare_data(X, features_to_use, target_col=None):
    """
    Подготовка данных для моделирования

    Parameters:
    X - исходные признаки
    features_to_use - список признаков для использования
    target_col - целевая переменная (если нужно исключить)
    """

    # Выбираем только нужные признаки
    if target_col and target_col in X.columns:
        X_selected = X.drop(target_col, axis=1)
    else:
        X_selected = X[features_to_use].copy()

    # Разделяем на категориальные и числовые
    cat_cols = [col for col in features_to_use if col in categorical_features]
    num_cols = [col for col in features_to_use if col in numeric_features]

    # Кодируем категориальные признаки
    X_encoded = pd.DataFrame()

    for col in cat_cols:
        if col in X_selected.columns:
            # OneHot кодирование для категориальных признаков
            encoder = OneHotEncoder(sparse_output=False, drop='first')
            encoded = encoder.fit_transform(X_selected[[col]])
            encoded_df = pd.DataFrame(encoded,
                                     columns=[f"{col}_{cat}" for cat in encoder.categories_[0][1:]],
                                     index=X_selected.index)
            X_encoded = pd.concat([X_encoded, encoded_df], axis=1)

    # Стандартизируем числовые признаки
    if num_cols:
        scaler = StandardScaler()
        X_scaled_num = scaler.fit_transform(X_selected[num_cols])
        X_scaled_num_df = pd.DataFrame(X_scaled_num,
                                       columns=[f"{col}_scaled" for col in num_cols],
                                       index=X_selected.index)
        X_encoded = pd.concat([X_encoded, X_scaled_num_df], axis=1)

    return X_encoded, scaler if num_cols else None

# Функция для расчета метрик
def calculate_metrics(y_true, y_pred, model_name=""):
    """Расчет метрик качества модели"""
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    # Рассчитываем MAPE (Mean Absolute Percentage Error)
    # Избегаем деления на ноль
    mask = y_true != 0
    if mask.any():
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan

    metrics = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    }

    print(f"\n{'='*50}")
    print(f"Метрики для {model_name}:")
    print(f"{'='*50}")
    print(f"MAE (Средняя абсолютная ошибка): {mae:.4f}")
    print(f"MSE (Среднеквадратичная ошибка): {mse:.4f}")
    print(f"RMSE (Корень из MSE): {rmse:.4f}")
    print(f"R² (Коэффициент детерминации): {r2:.4f}")
    print(f"MAPE (Средняя абсолютная процентная ошибка): {mape:.2f}%")

    return metrics

Часть 5: Моделирование
Модель 1: Все признаки на всей выборке

In [ ]:
print("="*60)
print("МОДЕЛЬ 1: Все признаки на всей выборке")
print("="*60)

# Подготовка всех признаков
all_features = categorical_features + numeric_features
X_all, scaler_all = prepare_data(X, all_features)

# Обучение модели
model1 = LinearRegression()
model1.fit(X_all, y)

# Предсказание
y_pred1 = model1.predict(X_all)

# Метрики
metrics1 = calculate_metrics(y, y_pred1, "Модель 1 (все признаки, вся выборка)")

# Визуализация коэффициентов
plt.figure(figsize=(12, 6))
coef_df1 = pd.DataFrame({
    'Признак': X_all.columns,
    'Коэффициент': model1.coef_
}).sort_values('Коэффициент', key=abs, ascending=False)

plt.barh(coef_df1['Признак'][:15], coef_df1['Коэффициент'][:15])
plt.xlabel('Значение коэффициента')
plt.title('Топ-15 коэффициентов модели 1 (все признаки)')
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nАнализ модели 1:")
print("1. Модель обучена на всей выборке без разделения.")
print("2. R² показывает, насколько хорошо модель объясняет данные.")
print(f"3. Самые важные признаки: {list(coef_df1['Признак'].head(5).values)}")
print("4. Риск переобучения, так как нет тестовой выборки.")

Модель 2: Все признаки с разделением на train/test

In [ ]:
print("\n" + "="*60)
print("МОДЕЛЬ 2: Все признаки с train/test split")
print("="*60)

# Разделение данных
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_all, y, test_size=0.2, random_state=42
)

print(f"Размеры данных:")
print(f"X_train: {X_train_all.shape}, X_test: {X_test_all.shape}")
print(f"y_train: {y_train_all.shape}, y_test: {y_test_all.shape}")

# Обучение модели
model2 = LinearRegression()
model2.fit(X_train_all, y_train_all)

# Предсказания
y_train_pred2 = model2.predict(X_train_all)
y_test_pred2 = model2.predict(X_test_all)

# Метрики для обучающей выборки
print("\nМетрики на обучающей выборке:")
metrics2_train = calculate_metrics(y_train_all, y_train_pred2, "Модель 2 (обучающая)")

# Метрики для тестовой выборки
print("\nМетрики на тестовой выборке:")
metrics2_test = calculate_metrics(y_test_all, y_test_pred2, "Модель 2 (тестовая)")

# Сравнение метрик
comparison_df = pd.DataFrame({
    'Обучающая': [metrics2_train['R²'], metrics2_train['RMSE'], metrics2_train['MAE']],
    'Тестовая': [metrics2_test['R²'], metrics2_test['RMSE'], metrics2_test['MAE']]
}, index=['R²', 'RMSE', 'MAE'])

print("\nСравнение метрик обучающей и тестовой выборок:")
print(comparison_df)

# Визуализация коэффициентов
plt.figure(figsize=(12, 6))
coef_df2 = pd.DataFrame({
    'Признак': X_all.columns,
    'Коэффициент': model2.coef_
}).sort_values('Коэффициент', key=abs, ascending=False)

plt.barh(coef_df2['Признак'][:15], coef_df2['Коэффициент'][:15])
plt.xlabel('Значение коэффициента')
plt.title('Топ-15 коэффициентов модели 2 (все признаки)')
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

# Визуализация предсказаний vs реальных значений
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Обучающая выборка
axes[0].scatter(y_train_all, y_train_pred2, alpha=0.6)
axes[0].plot([y_train_all.min(), y_train_all.max()],
             [y_train_all.min(), y_train_all.max()], 'r--', lw=2)
axes[0].set_xlabel('Реальные значения')
axes[0].set_ylabel('Предсказанные значения')
axes[0].set_title('Обучающая выборка: предсказания vs реальные значения')
axes[0].grid(True, alpha=0.3)

# Тестовая выборка
axes[1].scatter(y_test_all, y_test_pred2, alpha=0.6)
axes[1].plot([y_test_all.min(), y_test_all.max()],
             [y_test_all.min(), y_test_all.max()], 'r--', lw=2)
axes[1].set_xlabel('Реальные значения')
axes[1].set_ylabel('Предсказанные значения')
axes[1].set_title('Тестовая выборка: предсказания vs реальные значения')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nАнализ модели 2:")
print("1. Модель обучена с разделением на train/test (80/20).")
print("2. Разница в метриках между train и test показывает степень переобучения.")
print(f"3. R² на тесте: {metrics2_test['R²']:.4f} (хорошо, если > 0.7)")
print("4. Если R² на тесте значительно ниже, чем на train - признак переобучения.")

Модель 3: Лучшие признаки на всей выборке

In [ ]:
print("\n" + "="*60)
print("МОДЕЛЬ 3: Лучшие признаки на всей выборке")
print("="*60)

# Подготовка данных с лучшими признаками
X_best, scaler_best = prepare_data(X, best_features)

print(f"Используем {len(best_features)} лучших признаков:")
print(best_features)
print(f"\nРазмер X_best после обработки: {X_best.shape}")

# Обучение модели
model3 = LinearRegression()
model3.fit(X_best, y)

# Предсказание
y_pred3 = model3.predict(X_best)

# Метрики
metrics3 = calculate_metrics(y, y_pred3, "Модель 3 (лучшие признаки, вся выборка)")

# Визуализация коэффициентов
plt.figure(figsize=(10, 6))
coef_df3 = pd.DataFrame({
    'Признак': X_best.columns,
    'Коэффициент': model3.coef_
}).sort_values('Коэффициент', key=abs, ascending=False)

plt.barh(coef_df3['Признак'], coef_df3['Коэффициент'])
plt.xlabel('Значение коэффициента')
plt.title('Коэффициенты модели 3 (лучшие признаки)')
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nАнализ модели 3:")
print("1. Использованы только лучшие признаки, отобранные в EDA.")
print("2. Меньше признаков → проще интерпретировать модель.")
print(f"3. R²: {metrics3['R²']:.4f} (сравните с моделью 1: {metrics1['R²']:.4f})")
print("4. Если R² сопоставим с моделью 1, значит отбор признаков эффективен.")

Модель 4: Лучшие признаки с разделением на train/test

In [ ]:
print("\n" + "="*60)
print("МОДЕЛЬ 4: Лучшие признаки с train/test split")
print("="*60)

# Разделение данных
X_train_best, X_test_best, y_train_best, y_test_best = train_test_split(
    X_best, y, test_size=0.2, random_state=42
)

print(f"Размеры данных:")
print(f"X_train: {X_train_best.shape}, X_test: {X_test_best.shape}")
print(f"y_train: {y_train_best.shape}, y_test: {y_test_best.shape}")

# Обучение модели
model4 = LinearRegression()
model4.fit(X_train_best, y_train_best)

# Предсказания
y_train_pred4 = model4.predict(X_train_best)
y_test_pred4 = model4.predict(X_test_best)

# Метрики для обучающей выборки
print("\nМетрики на обучающей выборке:")
metrics4_train = calculate_metrics(y_train_best, y_train_pred4, "Модель 4 (обучающая)")

# Метрики для тестовой выборки
print("\nМетрики на тестовой выборке:")
metrics4_test = calculate_metrics(y_test_best, y_test_pred4, "Модель 4 (тестовая)")

# Сравнение всех моделей
print("\n" + "="*60)
print("СРАВНЕНИЕ ВСЕХ МОДЕЛЕЙ")
print("="*60)

comparison_all = pd.DataFrame({
    'Модель 1 (все, вся)': [metrics1['R²'], metrics1['RMSE'], metrics1['MAE']],
    'Модель 2 (все, тест)': [metrics2_test['R²'], metrics2_test['RMSE'], metrics2_test['MAE']],
    'Модель 3 (лучшие, вся)': [metrics3['R²'], metrics3['RMSE'], metrics3['MAE']],
    'Модель 4 (лучшие, тест)': [metrics4_test['R²'], metrics4_test['RMSE'], metrics4_test['MAE']]
}, index=['R²', 'RMSE', 'MAE'])

print(comparison_all)

# Визуализация сравнения моделей
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Сравнение R²
axes[0].bar(comparison_all.columns, comparison_all.loc['R²'])
axes[0].set_ylabel('R²')
axes[0].set_title('Сравнение R² для всех моделей')
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(comparison_all.loc['R²']):
    axes[0].text(i, v + 0.01, f'{v:.3f}', ha='center')

# Сравнение RMSE
axes[1].bar(comparison_all.columns, comparison_all.loc['RMSE'])
axes[1].set_ylabel('RMSE')
axes[1].set_title('Сравнение RMSE для всех моделей')
axes[1].tick_params(axis='x', rotation=45)
for i, v in enumerate(comparison_all.loc['RMSE']):
    axes[1].text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.show()

# Визуализация предсказаний для модели 4
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Обучающая выборка
axes[0].scatter(y_train_best, y_train_pred4, alpha=0.6)
axes[0].plot([y_train_best.min(), y_train_best.max()],
             [y_train_best.min(), y_train_best.max()], 'r--', lw=2)
axes[0].set_xlabel('Реальные значения')
axes[0].set_ylabel('Предсказанные значения')
axes[0].set_title('Модель 4: Обучающая выборка')
axes[0].grid(True, alpha=0.3)

# Тестовая выборка
axes[1].scatter(y_test_best, y_test_pred4, alpha=0.6)
axes[1].plot([y_test_best.min(), y_test_best.max()],
             [y_test_best.min(), y_test_best.max()], 'r--', lw=2)
axes[1].set_xlabel('Реальные значения')
axes[1].set_ylabel('Предсказанные значения')
axes[1].set_title('Модель 4: Тестовая выборка')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("ИТОГОВЫЕ ВЫВОДЫ И АНАЛИЗ")
print("="*60)

print("\n1. КАЧЕСТВО МОДЕЛЕЙ:")
print(f"   - Лучшая модель по R² на тесте: {comparison_all.loc['R²'].idxmax()} "
      f"({comparison_all.loc['R²'].max():.4f})")
print(f"   - Лучшая модель по RMSE: {comparison_all.loc['RMSE'].idxmin()} "
      f"({comparison_all.loc['RMSE'].min():.4f})")

print("\n2. АНАЛИЗ ПЕРЕОБУЧЕНИЯ:")
print(f"   - Модель 2 (все признаки): R²_train={metrics2_train['R²']:.4f}, "
      f"R²_test={metrics2_test['R²']:.4f}, разница={metrics2_train['R²']-metrics2_test['R²']:.4f}")
print(f"   - Модель 4 (лучшие признаки): R²_train={metrics4_train['R²']:.4f}, "
      f"R²_test={metrics4_test['R²']:.4f}, разница={metrics4_train['R²']-metrics4_test['R²']:.4f}")

print("\n3. ВЛИЯНИЕ ПРИЗНАКОВ:")
print("   На основе коэффициентов моделей, наиболее важные признаки для предсказания сна:")
print("   - Физическая активность (отрицательная корреляция)")
print("   - Уровень стресса (отрицательная корреляция)")
print("   - Возраст (отрицательная корреляция)")
print("   - Профессия (некоторые профессии связаны с лучшим/худшим сном)")

print("\n4. РЕКОМЕНДАЦИИ:")
print("   - Использовать модель с лучшими признаками (модель 4) для производства")
print("   - Дальнейшие улучшения: попробовать другие алгоритмы (деревья, ансамбли)")
print("   - Добавить полиномиальные признаки или взаимодействия")
print("   - Провести более тщательный отбор признаков (например, с помощью L1-регуляризации)")

Часть 6: Дополнительный анализ и интерпретация

In [ ]:
# Функция для интерпретации модели
def interpret_model(model, feature_names, top_n=10):
    """Интерпретация коэффициентов линейной регрессии"""

    coef_df = pd.DataFrame({
        'Признак': feature_names,
        'Коэффициент': model.coef_
    }).sort_values('Коэффициент', key=abs, ascending=False)

    print("\n" + "="*60)
    print("ИНТЕРПРЕТАЦИЯ КОЭФФИЦИЕНТОВ МОДЕЛИ")
    print("="*60)

    print("\nСамые важные положительные влияния (увеличивают продолжительность сна):")
    pos_coef = coef_df[coef_df['Коэффициент'] > 0].head(top_n//2)
    for _, row in pos_coef.iterrows():
        print(f"  {row['Признак']}: +{row['Коэффициент']:.4f}")

    print("\nСамые важные отрицательные влияния (уменьшают продолжительность сна):")
    neg_coef = coef_df[coef_df['Коэффициент'] < 0].head(top_n//2)
    for _, row in neg_coef.iterrows():
        print(f"  {row['Признак']}: {row['Коэффициент']:.4f}")

    return coef_df

# Интерпретируем лучшую модель (модель 4)
print("\nИнтерпретация лучшей модели (Модель 4):")
coef_interpretation = interpret_model(model4, X_best.columns)

# Пример предсказания для новых данных
print("\n" + "="*60)
print("ПРИМЕР ПРЕДСКАЗАНИЯ ДЛЯ НОВЫХ ДАННЫХ")
print("="*60)

# Создаем пример нового человека
example_data = {
    'Age': 35,
    'Physical Activity Level': 60,
    'Stress Level': 5,
    'Heart Rate': 72,
    'Daily Steps': 8000,
    'BMI Category': 'Normal',
    'Occupation': 'Software Engineer'
}

# Преобразуем в DataFrame
example_df = pd.DataFrame([example_data])

# Подготовим данные так же, как и для обучения
# Для простоты используем тот же scaler и encoder
example_prepared = prepare_data(example_df, best_features)[0]

# Делаем предсказание
predicted_sleep = model4.predict(example_prepared)[0]
print(f"\nДля человека с параметрами:")
for key, value in example_data.items():
    print(f"  {key}: {value}")
print(f"\nПредсказанная продолжительность сна: {predicted_sleep:.2f} часов")
print(f"Это {'выше' if predicted_sleep > y.mean() else 'ниже'} среднего ({y.mean():.2f} часов)")

# Анализ остатков для лучшей модели
print("\n" + "="*60)
print("АНАЛИЗ ОСТАТКОВ (РЕЗИДУАЛОВ)")
print("="*60)

# Вычисляем остатки для тестовой выборки
residuals = y_test_best - y_test_pred4

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Распределение остатков
axes[0].hist(residuals, bins=20, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Остатки (реальные - предсказанные)')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение остатков модели 4')
axes[0].grid(True, alpha=0.3)

# Остатки vs предсказанные значения
axes[1].scatter(y_test_pred4, residuals, alpha=0.6)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Предсказанные значения')
axes[1].set_ylabel('Остатки')
axes[1].set_title('Остатки vs предсказанные значения')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nАнализ остатков:")
print("1. Остатки должны быть случайно распределены вокруг нуля")
print("2. Не должно быть паттернов в графике 'остатки vs предсказанные'")
print("3. Распределение должно быть примерно нормальным")
print("4. Если есть паттерны, модель можно улучшить")

Итоговый отчет

In [ ]:
print("="*80)
print("ИТОГОВЫЙ ОТЧЕТ ПО ВЫПОЛНЕНИЮ ДОМАШНЕГО ЗАДАНИЯ")
print("="*80)

print("\n1. ВЫПОЛНЕННЫЕ ЭТАПЫ:")
print("   ✓ Загрузка и предобработка данных")
print("   ✓ Разведочный анализ (EDA)")
print("   ✓ Обработка пропусков (в данном датасете не требовалась)")
print("   ✓ Отбор признаков")
print("   ✓ Предобработка признаков (стандартизация, OneHot кодирование)")
print("   ✓ Обучение 4 моделей линейной регрессии")
print("   ✓ Оценка метрик качества")
print("   ✓ Интерпретация результатов")

print("\n2. КЛЮЧЕВЫЕ РЕЗУЛЬТАТЫ:")
print(f"   - Размер датасета: {df.shape}")
print(f"   - Целевая переменная: Sleep Duration (среднее: {y.mean():.2f} часов)")
print(f"   - Лучшая модель (R² на тесте): {comparison_all.loc['R²'].idxmax()} = {comparison_all.loc['R²'].max():.4f}")
print(f"   - Наиболее важные признаки: Age, Physical Activity Level, Stress Level")

print("\n3. ВЫВОДЫ ПО МОДЕЛЯМ:")
print("   - Модель на всех признаках показала хорошее качество, но может переобучаться")
print("   - Модель на отобранных признаках почти не уступает в качестве, но проще")
print("   - Разделение на train/test необходимо для оценки реальной эффективности")
print("   - Линейная регрессия хорошо подходит для данной задачи")

print("\n4. РЕКОМЕНДАЦИИ ДЛЯ УЛУЧШЕНИЯ:")
print("   - Попробовать полиномиальные признаки")
print("   - Использовать регуляризацию (Lasso, Ridge)")
print("   - Попробовать другие алгоритмы (случайный лес, градиентный бустинг)")
print("   - Собрать больше данных для улучшения обобщающей способности")

print("\n" + "="*80)